In [2]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
import scipy as sp
import time
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import io
from io import StringIO 
#import datedelta
import calendar
import  csv
import json
import random
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from scipy import stats
def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    print(df.shape)
    return df
def get_support(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                candle_size = .001,
                bound_pips = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])

        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if arr[i,reversal_col] == min_ and (i + lookup_range2 + stop_range < arr.shape[0]):
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            c = c + 1
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                #CANDLE BULL OR BEAR
                if arr[j,c_col] >= arr[j,o_col]:
                    bull = True
                else:
                    bull = False                        
                lower_bound = val - bound_pips
                upper_bound = val + bound_pips
                if i + lookup_range2 == j:
                    min_val = arr[j, reversal_col]
                else:
                    min_val = min(arr[i + lookup_range2 : j, reversal_col])
                    
                cur_val = arr[j - 1, reversal_col]

                if lookup_check == 0 and \
                bull == True and\
                arr[j,d_col] >= candle_size and\
                min_val >= lower_bound and \
                cur_val <= upper_bound and cur_val >= lower_bound:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = lookup_ind
                    lookup_check = 1
                    break

       # except:
        #    pass

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                candle_size = .001,
                bound_pips = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['resistance_lookup'] = 0
    df['resistance_indicator'] = 0

    lookup_col = df.columns.get_loc('resistance_lookup')
    new_col = df.columns.get_loc('resistance_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        max_ = max(arr[i - lookup_range : i + lookup_range2,reversal_col])

        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if arr[i,reversal_col] == max_ and (i + lookup_range2 + stop_range < arr.shape[0]):
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            c = c + 1
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                #CANDLE BULL OR BEAR
                if arr[j,c_col] >= arr[j,o_col]:
                    bull = True
                else:
                    bull = False                        
                lower_bound = val - bound_pips
                upper_bound = val + bound_pips
                if i + lookup_range2 == j:
                    max_val = arr[j, reversal_col]
                else:
                    max_val = max(arr[i + lookup_range2 : j, reversal_col])
                cur_val = arr[j - 1, reversal_col]

                if lookup_check == 0 and \
                bull == False and\
                arr[j,d_col] >= candle_size and\
                max_val <= upper_bound and \
                cur_val <= upper_bound and cur_val >= lower_bound:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = lookup_ind
                    lookup_check = 1
                    break

       # except:
        #    pass

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df





def run_single_pair_sim(pair,
                lookup_range,
                lookup_range2,
                stop_range,
                candle_size,
                bound_pips,
                sls,
                tps,
                spread):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl + spread:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl - spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses


    s = time.time()
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    df_ = df.copy()

    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'INSTRUMENT:',pair,
          'lookup_range',lookup_range,
          'lookup_range2', lookup_range2,
                    'stop_range', stop_range,
                    'candle_size' , candle_size,
                    'bound_pips' , bound_pips)

    df = get_support(df,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    candle_size = candle_size,
                    bound_pips = bound_pips
                    )

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    candle_size = candle_size,
                    bound_pips = bound_pips
                   )


    cols = ['instrument',
           'buy_support_val',
           'sell_resistance_val',
           'buy_resistance_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate2',
           'win_rate3',
           'win_rate4',
            'lookup_range',
            'lookup_range2',
            'stop_range',
            'candle_size',
            'bound_pips',
            'support_number_of_trades',
            'resistance_number_of_trades'
          ]

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


    for sl in sls:
        for tp in tps:
            if actual_number_of_trades == 0:
                ending_val,win_rate,wins,losses = 0,0,0,0
                ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
            else:
                ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   
                
            if actual_number_of_trades2 == 0:
                ending_val2,win_rate2,wins2,losses2 = 0,0,0,0
                ending_val3,win_rate3,wins3,losses3 = 0,0,0,0
            else:
                ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val3,win_rate3,wins3,losses3 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 
                
                
            max_ending_val = max(ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4)
            lst = [instrument,
                   ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4,
                   max_ending_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                   round(win_rate,4),
                   round(win_rate2,4),
                   round(win_rate3,4),
                   round(win_rate4,4),
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    candle_size,
                    bound_pips,
                   actual_number_of_trades,
                   actual_number_of_trades2
                  ]
            temp_lst.append(lst)

    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)
    
    e = time.time()
    best_buy_support = final_df['buy_support_val'].max()
    best_sell_resistance = final_df['sell_resistance_val'].max()
    best_buy_resistance = final_df['buy_resistance_val'].max()
    best_sell_support = final_df['sell_support_val'].max()
    max_val_all = final_df['max_ending_val'].max()
    print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell resistance:',
          round(best_sell_resistance),'||best buy resistance:',round(best_buy_resistance),'||best sell support:',round(best_sell_support))


    print('TOTAL SINGLE PAIR SIM TIME:',round((e-s)/60,1),'MINUTES')
    return final_df

def run_all_pairs_sim_loop(
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    candle_size ,
                    bound_pips,
                    spread):
    dir_name = os.getcwd() + '/reversal_counter_sim_results/' + str(lookup_range) + '_' + str(lookup_range2) + '_' + str(stop_range) + '_' + str(candle_size) + '_' + str(bound_pips)
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 

    if len(os.listdir(dir_name)) > 0:
        print('file already exists in this directory, passing...')
    else:
        print('file does not exist, running sim')


        all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


        final_df = pd.DataFrame([0])    
        for pair in all_pairs:
            print(pair)  

            if 'JPY' in pair:
                sls = [.05,.075,.1,.125,.15,.2,.25]
                tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
                spread = .01
                candle_size_ = candle_size * 100
                bound_pips_ = bound_pips * 100
            else:
                sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
                tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
                spread = .0001
                candle_size_ = candle_size 
                bound_pips_ = bound_pips        

            df = run_single_pair_sim(pair,
                            lookup_range,
                            lookup_range2,
                            stop_range,
                            candle_size_,
                            bound_pips_,
                            sls,
                            tps,
                            spread)    
            if final_df.shape[0] == 1:
                final_df = df
            else:
                final_df = pd.concat([final_df,df],axis = 0)

        final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)    

        gb = final_df.groupby('instrument').agg({'max_ending_val':max,'buy_support_val':max,'sell_resistance_val':max,'buy_resistance_val':max,'sell_support_val':max})
        gb = gb.reset_index()
        gb = gb.sort_values(by = 'max_ending_val',ascending = False)    
        for i in range(gb.shape[0]):
            print('pair',gb['instrument'].iloc[i],'||MAX VAL',gb['max_ending_val'].iloc[i],'||best buy support:',gb['buy_support_val'].iloc[i],'||best sell resistance:',
                  gb['sell_resistance_val'].iloc[i],'||best buy resistance:',gb['buy_resistance_val'].iloc[i],'||best sell support:',gb['sell_support_val'].iloc[i])

        print('MEDIANS:',np.median(gb['max_ending_val']),np.median(gb['buy_support_val']),np.median(gb['sell_resistance_val']),np.median(gb['buy_resistance_val']),np.median(gb['sell_support_val']))

        path = dir_name + '/' + str(round(np.median(gb['max_ending_val']))) + '_' + str(round(np.median(gb['buy_support_val']))) + '_' + str(round(np.median(gb['sell_resistance_val']))) + '_' + str(round(np.median(gb['buy_resistance_val']))) + '_' + str(round(np.median(gb['sell_support_val'])))
        path = path + '.csv'
        print('FINAL PATH:',path)
        print('SAVING DATA')
        final_df.to_csv(path,index = False)
        print()
        print()
        print()
        print()
        print()
        
def get_support(df,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['hour_col'] = 0
        df['xau_morning'] = 0
        new_col = df.columns.get_loc('hour_col')
        new_col2 = df.columns.get_loc('xau_morning')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).hour
            if arr[i,new_col] == 11:
                arr[i,new_col2] = 1
                
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    print(df.shape)
    return df
#pair = 'XAU_USD'
#df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
#print(df.shape)
#df.head()

In [7]:
test

(         next_hour  next_two_hours  next_three_hours  next_four_hours  \
 NZD_CHF        0.3             0.0              1.95              4.7   
 
          next_five_hours  next_six_hours  
 NZD_CHF             1.45             5.2  ,
          next_hour  next_two_hours  next_three_hours  next_four_hours  \
 NZD_CHF     0.6275            0.92            2.9825         5.425833   
 
          next_five_hours  next_six_hours  
 NZD_CHF           5.4025        5.375833  ,
          next_hour  next_two_hours  next_three_hours  next_four_hours  \
 NZD_CHF       75.3           110.4             357.9            651.1   
 
          next_five_hours  next_six_hours  
 NZD_CHF            648.3           645.1  )

In [11]:
t = test[0]
t = t.reset_index()
t.iloc[0,0] = t.iloc[0,0] + '_early'
t

,index,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours
0,NZD_CHF_early,0.3,0.0,1.95,4.7,1.45,5.2


In [194]:
def test_future5(df,pair,column = 'support_indicator'):
    """
    THIS FUNCTION CHECKS ALL TRADES
    
    """
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i - 1,c_col]
            
            min_2 = min(arr[i : i + 3,l_col]) 
            min_4 = min(arr[i : i + 4,l_col]) 
            min_6 = min(arr[i : i + 7,l_col]) 
            min_8 = min(arr[i : i + 9,l_col])            
            
            max_2 = max(arr[i : i + 3,h_col]) 
            max_4 = max(arr[i : i + 5,h_col]) 
            max_6 = max(arr[i : i + 7,h_col]) 
            max_8 = max(arr[i : i + 9,h_col]) 
            lst.append([pair,
                        mult * (arr[i ,c_col] - c),
                    mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 8,c_col] - c),
                    mult * (arr[i ,h_col] - c),
                    mult * (arr[i + 1,h_col] - c),
                        mult * (arr[i + 2,h_col] - c),
                        mult * (arr[i + 3,h_col] - c),
                        mult * (arr[i + 4,h_col] - c), 
                        mult * (arr[i + 5,h_col] - c), 
                        mult * (arr[i + 6,h_col] - c),
                        mult * (arr[i + 8,h_col] - c),
                        mult * (c - arr[i ,l_col]),
                        mult * (c - arr[i + 1,l_col]),
                        mult * (c - arr[i + 2,l_col] ),
                        mult * (c - arr[i + 3,l_col] ),
                        mult * (c - arr[i + 4,l_col] ), 
                        mult * (c - arr[i + 5,l_col] ), 
                        mult * (c - arr[i + 6,l_col] ),
                        mult * (c - arr[i + 7,l_col] ),
                        mult * (c - arr[i + 8,l_col]),
                        mult * (c - min_2),
                        mult * (c - min_4),
                        mult * (c - min_6),
                        mult * (c - min_8),
                        mult * (max_2 - c ),
                        mult * (max_4 - c),
                        mult * (max_6 - c ),
                        mult * (max_8 - c),
                       ])
            
      
    df2 = pd.DataFrame(lst,columns = ['pair','same_hour','next_hour','next_two_hours','next_three_hours',
                                      'next_four_hours','next_five_hours','next_six_hours','next_eight_hours'
                                      ,'same_hour_h','next_hour_h','next_two_hours_h','next_three_hours_h',
                                      'next_four_hours_h','next_five_hours_h','next_six_hours_h','next_eight_hours_h'
                                      ,'same_hour_dd','next_hour_dd','next_two_hours_dd','next_three_hours_dd',
                                      'next_four_hours_dd','next_five_hours_dd','next_six_hours_dd','next_seven_hours_dd','next_eight_hours_dd',
                                      'max_dd2','max_dd4','max_dd6','max_dd8','max_upside2','max_upside4','max_upside6','max_upside8'
                                     ])  

    
    return df2
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
medians = pd.DataFrame([-100])
means = pd.DataFrame([-100])
sums = pd.DataFrame([-100])
final_df = pd.DataFrame([-100])
for pair in all_pairs:


    print(pair,'LATER DATES')
    #df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df = load_df(pair = pair,granularity ='H1',start = datetime(2010,1,1,0,0,0),end = datetime(2016,1,1,0,0,0))
    df = get_support(df,pair)
    df_ = test_future5(df,pair,column = 'support_indicator')
    if final_df.iloc[0,0] == -100:
        final_df = df_
    else:
        final_df = pd.concat([final_df,df_])
final_df

USD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_H1/2010-01-01_2016-01-01.csv
(39068, 15)
SHAPE (24, 16)
EUR_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2010-01-01_2016-01-01.csv
(39094, 15)
SHAPE (89, 16)
AUD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2010-01-01_2016-01-01.csv
(39082, 15)
SHAPE (61, 16)
CAD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2010-01-01_2016-01-01.csv
(39049, 15)
SHAPE (64, 16)
CHF_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CHF_JPY_H1/2010-01-01_2016-01-01.csv
(39048, 15)
SHAPE (60, 16)
GBP_USD LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_USD_H1/2010-01-01_2016-01-01.csv
(39062, 15)
SHAPE (126, 16)
USD_CAD LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_CA

,pair,same_hour,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours,next_eight_hours,same_hour_h,...,next_seven_hours_dd,next_eight_hours_dd,max_dd2,max_dd4,max_dd6,max_dd8,max_upside2,max_upside4,max_upside6,max_upside8
0,USD_JPY,8.4,-16.2,-29.0,-45.0,-72.2,-66.0,-82.8,-64.2,22.8,...,83.8,76.6,29.8,51.0,91.0,91.0,22.8,22.8,22.8,22.8
1,USD_JPY,6.2,-10.6,0.0,8.0,4.0,6.1,22.2,25.0,15.0,...,-13.6,-19.0,18.8,18.8,18.8,18.8,15.0,15.0,22.7,43.8
2,USD_JPY,10.6,26.8,12.4,13.1,14.4,18.8,23.0,25.8,15.6,...,-9.0,-9.0,1.3,1.3,1.3,1.3,30.0,30.0,30.0,34.8
3,USD_JPY,-32.0,-57.2,-53.4,-44.0,-44.4,-70.2,-83.6,-77.8,4.0,...,84.2,90.8,65.7,65.7,89.4,90.8,4.0,4.0,4.0,4.0
4,USD_JPY,-11.3,-13.4,-9.7,-26.3,-19.8,-9.8,-16.0,-20.1,7.3,...,18.7,20.9,27.9,31.5,35.5,35.5,7.3,7.3,7.3,7.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,NZD_CHF,-12.9,-25.2,3.7,27.6,70.1,74.0,61.5,20.9,16.8,...,-30.1,-17.0,30.1,30.1,30.1,30.1,16.8,79.5,97.0,97.0
73,NZD_CHF,18.7,35.3,37.7,51.9,57.3,68.7,43.1,47.1,28.9,...,-35.3,-34.6,2.6,2.6,2.6,2.6,52.7,62.5,79.3,79.3
74,NZD_CHF,4.6,11.8,20.2,28.4,51.4,67.0,76.7,55.0,23.6,...,-57.7,-50.1,9.2,9.2,9.2,9.2,23.6,59.2,81.2,81.2
75,NZD_CHF,4.0,-8.2,-1.2,1.0,-11.0,27.8,29.4,47.1,12.4,...,-14.2,-14.6,16.8,16.8,18.6,18.6,12.4,12.4,42.8,49.8


In [108]:
final_df[['same_hour','next_hour','next_two_hours','next_three_hours',
                                      'next_four_hours','next_five_hours','next_six_hours','next_eight_hours']].median()

same_hour           2.75
next_hour           4.00
next_two_hours      4.55
next_three_hours    4.60
next_four_hours     4.40
next_five_hours     5.75
next_six_hours      7.00
next_eight_hours    7.20
dtype: float64

In [113]:
df_ = final_df
spread = 1
for sl in [5,7,9,12,15,18,20,25]:
    print('SL',sl)
    loss = df_[df_['max_dd8'] >= sl - spread]
    win = df_[df_['max_dd8'] < sl - spread]
    print(loss.shape,win.shape)
    
    win_pips = round(win['next_eight_hours'].sum())
    loss_pips = round(loss.shape[0] * sl) + round(loss.shape[0] * spread)
    print('NET PIPS:',round(win_pips - loss_pips ),win_pips,loss_pips)

SL 5
(1224, 25) (186, 25)
NET PIPS: -400 6944 7344
SL 7
(1158, 25) (252, 25)
NET PIPS: 47 9311 9264
SL 9
(1087, 25) (323, 25)
NET PIPS: 806 11676 10870
SL 12
(1003, 25) (407, 25)
NET PIPS: 1339 14378 13039
SL 15
(914, 25) (496, 25)
NET PIPS: 1829 16453 14624
SL 18
(842, 25) (568, 25)
NET PIPS: 2274 18272 15998
SL 20
(790, 25) (620, 25)
NET PIPS: 2978 19568 16590
SL 25
(670, 25) (740, 25)
NET PIPS: 3422 20842 17420


In [117]:
win[win['max_upside4']==win['max_upside8']].shape

(280, 25)

In [118]:
win.shape

(740, 25)

In [201]:
df_ = final_df
spread = 1
mult = 1.5
for sl in [5,7,9,12,15,18,20,25]:
    print('SL',sl)
    loss = df_[df_['max_dd8'] >= sl - spread]
    win = df_[df_['max_dd8'] < sl - spread]
    win['max_drawdown'] = win[['next_five_hours_dd','next_six_hours_dd','next_seven_hours_dd','next_eight_hours_dd']].max(axis = 1)

    print(loss.shape,win.shape)
    win_pips = (mult * win[win['max_upside8'] >= (win['max_upside4'] * mult)]['max_upside4'].sum()) + win[win['max_upside8'] < (win['max_upside4'] * mult)]['next_eight_hours'].sum()
    
    #win_pips = round(win['next_eight_hours'].sum())
    loss_pips = round(loss.shape[0] * sl) + round(loss.shape[0] * spread)
    print('NET PIPS:',round(win_pips - loss_pips ),win_pips,loss_pips)
    
    

SL 5
(1229, 34) (181, 35)
NET PIPS: -492 6882.3000000000175 7374
SL 7
(1167, 34) (243, 35)
NET PIPS: -187 9149.30000000002 9336
SL 9
(1097, 34) (313, 35)
NET PIPS: 611 11581.050000000012 10970
SL 12
(1017, 34) (393, 35)
NET PIPS: 993 14214.250000000018 13221
SL 15
(937, 34) (473, 35)
NET PIPS: 1549 16541.20000000002 14992
SL 18
(864, 34) (546, 35)
NET PIPS: 1936 18351.65000000003 16416
SL 20
(813, 34) (597, 35)
NET PIPS: 2506 19578.800000000036 17073
SL 25
(700, 34) (710, 35)
NET PIPS: 3060 21259.50000000004 18200


<ipython-input-201-78f3385e4b8d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win['max_drawdown'] = win[['next_five_hours_dd','next_six_hours_dd','next_seven_hours_dd','next_eight_hours_dd']].max(axis = 1)
<ipython-input-201-78f3385e4b8d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win['max_drawdown'] = win[['next_five_hours_dd','next_six_hours_dd','next_seven_hours_dd','next_eight_hours_dd']].max(axis = 1)
<ipython-input-201-78f3385e4b8d>:8: SettingWithCopyWarning: 
A value is trying to be set on 

In [179]:
loss[loss['max_dd4'] < 25].shape

(348, 33)

In [190]:
loss[loss['next_four_hours'] > 0].shape

(272, 33)

In [183]:
loss.shape

(995, 33)

In [192]:
win.shape

(1079, 33)

In [211]:
loss[loss['next_four_hours'] > 0].shape
win[win['next_four_hours'] < 0].shape

(103, 35)

In [203]:
win.head()

,pair,same_hour,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours,next_eight_hours,same_hour_h,...,next_eight_hours_dd,max_dd2,max_dd4,max_dd6,max_dd8,max_upside2,max_upside4,max_upside6,max_upside8,max_drawdown
1,USD_JPY,6.2,-10.6,0.0,8.0,4.0,6.1,22.2,25.0,15.0,...,-19.0,18.8,18.8,18.8,18.8,15.0,15.0,22.7,43.8,-2.7
2,USD_JPY,10.6,26.8,12.4,13.1,14.4,18.8,23.0,25.8,15.6,...,-9.0,1.3,1.3,1.3,1.3,30.0,30.0,30.0,34.8,-9.0
5,USD_JPY,2.2,1.1,3.2,5.6,5.3,13.7,23.3,19.3,2.2,...,-13.3,1.5,1.5,5.1,5.1,4.1,12.2,30.3,30.3,-4.7
6,USD_JPY,10.3,16.9,7.2,7.7,17.9,1.7,9.7,8.8,10.8,...,-1.4,4.3,4.3,4.3,4.3,19.6,23.3,23.3,23.3,1.3
7,USD_JPY,20.8,18.0,24.2,22.8,28.2,22.8,28.2,23.4,36.8,...,-21.3,-0.2,-0.2,-0.2,-0.2,36.8,40.0,40.0,40.0,-20.0


In [212]:
win[win['next_four_hours'] < 0]['next_eight_hours'].mean()

7.8815533980583945

In [214]:
loss[(loss['next_four_hours'] < 0) & (loss['max_dd4'] < 25)]['next_four_hours'].mean()

-16.093382352941163

In [215]:
win[win['next_four_hours'] < 0]['next_eight_hours'].mean()

7.8815533980583945

In [216]:
win[win['next_six_hours'] < 0]['next_eight_hours'].mean()

1.1537500000002188

In [217]:
win[win['next_six_hours'] < 0]['next_eight_hours'].shape

(80,)

In [218]:
loss[(loss['next_six_hours'] < 0) & (loss['max_dd6'] < 25)]['next_six_hours'].mean()

-9.599999999999936

In [219]:
loss[(loss['next_six_hours'] < 0) & (loss['max_dd6'] < 25)].shape

(33, 34)

In [222]:
loss_shape = loss[(loss['next_four_hours'] < 0) & (loss['max_dd4'] < 25)].shape
win_shape = win[(win['next_four_hours'] < 0) & (win['max_dd4'] < 25)].shape
loss_result = loss[(loss['next_four_hours'] < 0) & (loss['max_dd4'] < 25)]['next_four_hours'].mean()
win_result = win[win['next_four_hours'] < 0]['next_eight_hours'].mean()
print(win_shape,loss_shape)
print(win_result,loss_result)

(103, 35) (136, 34)
7.8815533980583945 -16.093382352941163


In [224]:
win_result = win[win['next_four_hours'] > 0]['next_eight_hours'].mean()
win_result

34.61502504173627

In [221]:
win[(win['next_four_hours'] < 0) & (win['max_dd4'] < 25)].shape

(103, 35)

In [189]:
win[win['next_four_hours'] > 0]['next_eight_hours'].mean()

33.043049327354304

In [208]:
win[(win['next_four_hours'] > 0) & (win['max_drawdown'] > 0)]['next_eight_hours'].mean()

13.474316939890693

In [200]:
win[['next_five_hours_dd','next_six_hours_dd','next_seven_hours_dd','next_eight_hours_dd']].min(axis = 1)

1    -19.0
2    -15.6
5    -16.9
6     -7.0
7    -22.6
      ... 
69   -93.4
73   -56.5
74   -65.5
75   -14.6
76   -17.5
Length: 710, dtype: float64

In [161]:
final_df.head()

,pair,same_hour,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours,next_eight_hours,same_hour_h,...,next_six_hours_dd,next_eight_hours_dd,max_dd2,max_dd4,max_dd6,max_dd8,max_upside2,max_upside4,max_upside6,max_upside8
0,USD_JPY,7.3,11.8,7.8,-3.3,-18.5,-44.6,-87.1,-62.0,11.1,...,98.4,67.2,23.5,23.5,45.7,98.4,26.6,26.6,26.6,26.6
1,USD_JPY,18.8,27.8,45.7,56.5,66.2,74.3,77.8,121.0,29.9,...,-68.8,-89.2,0.0,0.0,0.0,0.0,29.9,66.1,84.2,100.4
2,USD_JPY,8.8,39.2,56.4,20.9,38.8,8.5,-6.1,-70.4,18.1,...,41.8,107.6,-0.2,-0.2,5.5,84.1,55.0,57.4,57.4,57.4
3,USD_JPY,-46.2,-31.9,-50.4,-74.5,-79.0,-105.4,-120.2,-81.7,6.0,...,138.2,183.4,56.2,83.2,112.0,174.5,6.0,6.0,6.0,6.0
4,USD_JPY,-24.1,-28.6,-55.0,-69.8,-122.6,-31.3,-57.3,-69.7,8.2,...,58.1,71.0,52.1,87.8,133.0,133.0,8.2,8.2,8.2,8.2


In [230]:
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
medians = pd.DataFrame([-100])
means = pd.DataFrame([-100])
sums = pd.DataFrame([-100])
final_df = pd.DataFrame([-100])
for pair in all_pairs:


    print(pair,'LATER DATES')
    df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
   # df = load_df(pair = pair,granularity ='H1',start = datetime(2010,1,1,0,0,0),end = datetime(2016,1,1,0,0,0))
    df = get_support(df,pair)
    df_ = test_future5(df,pair,column = 'support_indicator')
    if final_df.iloc[0,0] == -100:
        final_df = df_
    else:
        final_df = pd.concat([final_df,df_])

for pair in all_pairs:


    print(pair,'LATER DATES')
    #df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df = load_df(pair = pair,granularity ='H1',start = datetime(2010,1,1,0,0,0),end = datetime(2016,1,1,0,0,0))
    df = get_support(df,pair)
    df_ = test_future5(df,pair,column = 'support_indicator')
    if final_df.iloc[0,0] == -100:
        final_df = df_
    else:
        final_df = pd.concat([final_df,df_])

USD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_H1/2016-01-01_2022-07-31.csv
(41169, 15)
SHAPE (34, 16)
EUR_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2016-01-01_2022-07-31.csv
(41144, 15)
SHAPE (48, 16)
AUD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2016-01-01_2022-07-31.csv
(41163, 15)
SHAPE (29, 16)
CAD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2016-01-01_2022-07-31.csv
(41154, 15)
SHAPE (36, 16)
CHF_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CHF_JPY_H1/2016-01-01_2022-07-31.csv
(41154, 15)
SHAPE (30, 16)
GBP_USD LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_USD_H1/2016-01-01_2022-07-31.csv
(41155, 15)
SHAPE (51, 16)
USD_CAD LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_CAD

In [253]:
def sim_results(df,sl,risk_size = .01):
    balance = 10000
    spread = 1
    mult = 2.5
    for i in range(df.shape[0]):

        if df['max_dd8'].iloc[i] >= sl - spread:
            balance -= balance * risk_size
        elif df['max_upside8'].iloc[i] >= df['max_upside4'].iloc[i] * mult:
            balance += (balance * risk_size) * (df['max_upside4'].iloc[i] * mult) / (sl + spread)
        else:
            balance += (balance * risk_size) * (df['next_eight_hours'].iloc[i]) / (sl + spread)
        if i % 100 == 0:
            print(i)
            print(balance)
    print('ENDING BALANCE:',balance)
    
    
def sim_results(df,sl,risk_size = .01):
    balance = 10000
    spread = 1
    mult = 4
    for i in range(df.shape[0]):

        if df['max_dd8'].iloc[i] >= sl - spread:
            balance -= balance * risk_size
        elif df['max_upside8'].iloc[i] >= sl * mult:
            balance += (balance * risk_size) * (sl * mult) / (sl + spread)
        else:
            balance += (balance * risk_size) * (df['next_eight_hours'].iloc[i]) / (sl + spread)
    print('ENDING BALANCE:',balance)
    
    
def sim_results(df,sl,mult,risk_size = .01):
    balance = 10000
    spread = 1
   #mult = 2.5
    for i in range(df.shape[0]):

        if df['max_dd8'].iloc[i] >= sl - spread:
            balance -= balance * risk_size
        elif df['max_upside8'].iloc[i] >= df['max_upside4'].iloc[i] * mult:
            balance += (balance * risk_size) * (df['max_upside4'].iloc[i] * mult) / (sl + spread)
        else:
            balance += (balance * risk_size) * (df['next_eight_hours'].iloc[i]) / (sl + spread)
    print('ENDING BALANCE:',balance)   
    
def units_calc(trading_account,max_risk,sl):
    units=round((max_risk * trading_account / sl) * 10000)
    return units

def sim_results(df,sl,mult,risk_size = .01):
    balance = 10000
    spread = 1.5
   #mult = 2.5



    for i in range(df.shape[0]):
        max_h = df[['next_three_hours_h', 'next_four_hours_h', 'next_five_hours_h',
       'next_six_hours_h', 'next_eight_hours_h']].iloc[i,:].max()
        max_dd = df[[ 'next_five_hours_dd',
       'next_six_hours_dd','next_seven_hours_dd', 'next_eight_hours_dd']].iloc[i,:].max()    
        max_dd2 = df[[ 'next_seven_hours_dd', 'next_eight_hours_dd']].iloc[i,:].max()    
        
      #  if df['max_dd6'].iloc[i] < sl - spread and df['next_six_hours'].iloc[i] > 0 and max_dd >= 0:
       #     balance = balance
        if df['max_dd4'].iloc[i] < sl - spread and df['next_four_hours'].iloc[i] > 0 and max_dd >= 0:
            balance = balance
        elif df['max_upside4'].iloc[i] < 0 and df['max_dd4'].iloc[i] < sl - spread:
            balance += (balance * risk_size) * 10 / (sl + spread)
       # if df['next_six_hours'].iloc[i] < 0 and df['max_dd6'].iloc[i] < sl - spread:
        #    balance += (df['next_six_hours'].iloc[i] / sl) * balance * risk_size
        elif df['max_dd8'].iloc[i] >= sl - spread:
            balance -= balance * risk_size

        elif max_h >= df['max_upside4'].iloc[i] * mult and df['max_upside4'].iloc[i] > 0:
            balance += (balance * risk_size) * (df['max_upside4'].iloc[i] * mult) / (sl + spread)

        else:
            balance += (balance * risk_size) * (df['next_eight_hours'].iloc[i]) / (sl + spread)
        units = units_calc(trading_account = balance,max_risk = .01,sl = sl)
        units = units / 100000
        pricing = units * 5
        #balance -= pricing
    print('ENDING BALANCE:',balance)       
    
    
final_df2 = pd.concat([final_df,final_df])


for sl in [20,25,30]:
    for mult in [1.5,2,2.5,3]:
        print()
        print('SL:',sl,'MULT:',mult)
        sim_results(final_df2,sl = sl,mult = mult,risk_size = .01)


SL: 20 MULT: 1.5
ENDING BALANCE: 25656.865928366242

SL: 20 MULT: 2
ENDING BALANCE: 34736.57101797891

SL: 20 MULT: 2.5
ENDING BALANCE: 40543.95837636921

SL: 20 MULT: 3
ENDING BALANCE: 40283.89153181314

SL: 25 MULT: 1.5
ENDING BALANCE: 47166.383169023255

SL: 25 MULT: 2
ENDING BALANCE: 54899.56889191607

SL: 25 MULT: 2.5
ENDING BALANCE: 63773.38915454439

SL: 25 MULT: 3
ENDING BALANCE: 64001.17545305536

SL: 30 MULT: 1.5
ENDING BALANCE: 33289.49417055509

SL: 30 MULT: 2
ENDING BALANCE: 39317.354562784494

SL: 30 MULT: 2.5
ENDING BALANCE: 46704.511605971355

SL: 30 MULT: 3
ENDING BALANCE: 47464.43559863279


In [247]:
final_df[final_df['next_eight_hours'] > 0]['next_eight_hours'].mean()

34.20503778337536

In [248]:
final_df[final_df['next_four_hours'] > 0]['next_four_hours'].mean()

26.767525773195906

In [249]:
def units_calc(trading_account,max_risk,sl):
    units=round((max_risk * trading_account / sl) * 10000)
    print('MAX RISK:',max_risk,'SL:',sl,'LOTS:',round(units / 100000,2),'UNITS:',units)
    return units
units = units_calc(trading_account = 100000,max_risk = .01,sl = 25)
units

MAX RISK: 0.01 SL: 25 LOTS: 4.0 UNITS: 400000


400000

In [233]:
final_df2.shape[0]/10

414.8

In [244]:
def sim_results(df,sl,mult,risk_size = .01):
    balance = 100000
    spread = 1.5
   #mult = 2.5



    for i in range(df.shape[0]):
        max_h = df[['next_three_hours_h', 'next_four_hours_h', 'next_five_hours_h',
       'next_six_hours_h', 'next_eight_hours_h']].iloc[i,:].max()
        max_dd = df[[ 'next_five_hours_dd',
       'next_six_hours_dd','next_seven_hours_dd', 'next_eight_hours_dd']].iloc[i,:].max()        
        
        if df['max_dd8'].iloc[i] >= sl - spread:
            balance -= balance * risk_size

        elif max_h >= df['max_upside4'].iloc[i] * mult and df['max_upside4'].iloc[i] > 0:
            balance += (balance * risk_size) * (df['max_upside4'].iloc[i] * mult) / (sl + spread)
        else:
            balance += (balance * risk_size) * (df['next_eight_hours'].iloc[i]) / (sl + spread)
        if i % 400 == 0:
            print('YEAR BAL:',balance)
    print('ENDING BALANCE:',balance)       
    
    
final_df2 = pd.concat([final_df,final_df])


for sl in [15,20,25,30]:
    for mult in [1.5,2,2.5,3]:
        print()
        print('SL:',sl,'MULT:',mult)
        sim_results(final_df2,sl = sl,mult = mult,risk_size = .01)


SL: 15 MULT: 1.5
YEAR BAL: 99000.0
YEAR BAL: 102538.59942565487
YEAR BAL: 59054.81368982652
YEAR BAL: 45961.59743315103
YEAR BAL: 55818.914792968986
YEAR BAL: 56872.83631803045
YEAR BAL: 61983.99222158826
YEAR BAL: 53827.67567019382
YEAR BAL: 34271.559794908455
YEAR BAL: 48863.75616706279
YEAR BAL: 37780.516077505155
ENDING BALANCE: 57967.39107950132

SL: 15 MULT: 2
YEAR BAL: 99000.0
YEAR BAL: 105004.75635077545
YEAR BAL: 60653.938450520895
YEAR BAL: 42776.16139557588
YEAR BAL: 52734.09299911517
YEAR BAL: 56047.247122477944
YEAR BAL: 64018.79717283624
YEAR BAL: 54850.768037089845
YEAR BAL: 32505.568982096924
YEAR BAL: 46268.760932298464
YEAR BAL: 38765.170495080434
ENDING BALANCE: 57443.89186710695

SL: 15 MULT: 2.5
YEAR BAL: 99000.0
YEAR BAL: 100095.0589824949
YEAR BAL: 59278.08665787029
YEAR BAL: 42071.660401175584
YEAR BAL: 53198.41854414116
YEAR BAL: 61066.12490641078
YEAR BAL: 67779.29050149421
YEAR BAL: 59541.63035932899
YEAR BAL: 35147.5394087736
YEAR BAL: 50855.293842388026
YE

In [173]:
final_df.columns

Index(['pair', 'same_hour', 'next_hour', 'next_two_hours', 'next_three_hours',
       'next_four_hours', 'next_five_hours', 'next_six_hours',
       'next_eight_hours', 'same_hour_h', 'next_hour_h', 'next_two_hours_h',
       'next_three_hours_h', 'next_four_hours_h', 'next_five_hours_h',
       'next_six_hours_h', 'next_eight_hours_h', 'same_hour_dd',
       'next_hour_dd', 'next_two_hours_dd', 'next_three_hours_dd',
       'next_four_hours_dd', 'next_five_hours_dd', 'next_six_hours_dd',
       'next_eight_hours_dd', 'max_dd2', 'max_dd4', 'max_dd6', 'max_dd8',
       'max_upside2', 'max_upside4', 'max_upside6', 'max_upside8'],
      dtype='object')

In [175]:
final_df[['next_three_hours_h', 'next_four_hours_h', 'next_five_hours_h',
       'next_six_hours_h', 'next_eight_hours_h']].iloc[0,:].max()

14.19999999999959

In [136]:
2100 % 100

0

In [115]:
win = win.sort_values(by = 'max_upside8',ascending = False)
win.head(20)

,pair,same_hour,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours,next_eight_hours,same_hour_dd,...,next_six_hours_dd,next_eight_hours_dd,max_dd2,max_dd4,max_dd6,max_dd8,max_upside2,max_upside4,max_upside6,max_upside8
91,EUR_AUD,13.6,13.6,67.2,40.3,62.9,159.8,155.6,150.1,0.8,...,-151.3,-123.3,0.8,0.8,0.8,0.8,23.0,77.8,160.0,187.4
23,NZD_USD,32.0,27.4,9.1,22.4,92.4,116.1,125.4,120.6,13.7,...,-99.6,-95.6,13.7,13.7,13.7,13.7,45.5,45.5,135.1,186.4
43,EUR_CHF,64.7,56.2,70.8,105.2,143.8,145.4,160.2,171.7,2.7,...,-145.8,-168.1,2.7,2.7,2.7,2.7,112.9,114.4,180.9,180.9
13,CAD_CHF,34.9,52.8,84.6,108.2,148.8,167.0,166.0,151.6,2.6,...,-161.6,-144.8,2.6,2.6,2.6,2.6,56.8,120.6,172.4,176.4
32,EUR_JPY,38.6,71.7,125.9,119.2,117.7,124.6,152.0,192.2,7.8,...,-119.9,-152.6,7.8,7.8,7.8,7.8,72.0,132.9,132.9,174.0
17,EUR_CAD,110.5,103.4,101.8,127.8,131.8,125.2,148.6,128.3,4.3,...,-118.2,-124.4,4.3,4.3,4.3,4.3,134.6,134.6,145.2,172.1
97,EUR_AUD,12.7,43.0,25.4,-5.8,132.7,133.6,139.2,130.0,7.1,...,-134.4,-125.0,7.1,14.1,14.1,14.1,48.8,48.8,166.2,170.2
105,EUR_CAD,11.2,8.9,19.0,30.0,30.5,36.0,116.6,155.8,0.2,...,-35.8,-138.7,0.6,0.6,1.8,1.8,25.0,32.6,51.3,169.2
11,EUR_USD,2.0,39.0,69.2,95.2,144.2,132.4,134.0,140.6,2.6,...,-124.8,-125.0,9.0,9.0,9.0,9.0,47.4,99.8,164.4,164.4
105,GBP_USD,49.7,74.1,110.4,101.7,106.1,129.4,144.1,135.9,2.3,...,-129.4,-108.5,2.3,2.3,2.3,2.3,85.0,126.4,140.2,158.2


In [111]:
df_ = final_df
spread = 1
for sl in [5,7,9,12,15,18,20,25]:
    print('SL',sl)
    loss = df_[df_['max_dd4'] >= sl - spread]
    win = df_[df_['max_dd4'] < sl - spread]
    print(loss.shape,win.shape)
    win_pips = round(win['next_five_hours'].sum())
    loss_pips = round(loss.shape[0] * sl) + round(loss.shape[0] * spread)
    print('NET PIPS:',round(win_pips - loss_pips ),win_pips,loss_pips)

SL 5
(1167, 25) (243, 25)
NET PIPS: -123 6879 7002
SL 7
(1080, 25) (330, 25)
NET PIPS: 420 9060 8640
SL 9
(992, 25) (418, 25)
NET PIPS: 1080 11000 9920
SL 12
(892, 25) (518, 25)
NET PIPS: 1171 12767 11596
SL 15
(781, 25) (629, 25)
NET PIPS: 1903 14399 12496
SL 18
(690, 25) (720, 25)
NET PIPS: 2514 15624 13110
SL 20
(626, 25) (784, 25)
NET PIPS: 3081 16227 13146
SL 25
(502, 25) (908, 25)
NET PIPS: 3611 16663 13052


In [112]:
win

,pair,same_hour,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours,next_eight_hours,same_hour_dd,...,next_six_hours_dd,next_eight_hours_dd,max_dd2,max_dd4,max_dd6,max_dd8,max_upside2,max_upside4,max_upside6,max_upside8
1,USD_JPY,6.2,-10.6,0.0,8.0,4.0,6.1,22.2,25.0,8.8,...,-5.2,-19.0,18.8,18.8,18.8,18.8,15.0,15.0,15.0,24.8
2,USD_JPY,10.6,26.8,12.4,13.1,14.4,18.8,23.0,25.8,1.3,...,-15.6,-9.0,1.3,1.3,1.3,1.3,30.0,30.0,30.0,30.0
5,USD_JPY,2.2,1.1,3.2,5.6,5.3,13.7,23.3,19.3,0.3,...,-11.1,-13.3,1.5,1.5,5.1,5.1,2.4,12.2,24.1,30.3
6,USD_JPY,10.3,16.9,7.2,7.7,17.9,1.7,9.7,8.8,4.3,...,1.3,-1.4,4.3,4.3,4.3,4.3,19.6,19.6,23.3,23.3
7,USD_JPY,20.8,18.0,24.2,22.8,28.2,22.8,28.2,23.4,-0.2,...,-22.6,-21.3,-0.2,-0.2,-0.2,-0.2,36.8,40.0,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NZD_CHF,14.8,40.2,18.3,-7.8,-2.7,-41.9,-86.4,-90.2,4.8,...,104.2,90.9,4.8,7.9,77.7,104.2,41.8,43.6,43.6,43.6
73,NZD_CHF,18.7,35.3,37.7,51.9,57.3,68.7,43.1,47.1,2.6,...,-41.4,-34.6,2.6,2.6,2.6,2.6,42.5,62.1,79.3,79.3
74,NZD_CHF,4.6,11.8,20.2,28.4,51.4,67.0,76.7,55.0,7.4,...,-65.5,-50.1,9.2,9.2,9.2,9.2,23.6,46.8,72.8,81.2
75,NZD_CHF,4.0,-8.2,-1.2,1.0,-11.0,27.8,29.4,47.1,-0.1,...,2.4,-14.6,13.2,16.8,18.6,18.6,12.4,12.4,28.0,42.8


In [120]:
win[win['max_upside8'] >= (win['max_upside4'] * 2)].shape

(122, 25)

In [125]:
win[win['max_upside8'] >= (win['max_upside4'] * 1.5)]['max_upside8'].sum()

12698.299999999992

In [126]:
1.5 * win[win['max_upside8'] >= (win['max_upside4'] * 1.5)]['max_upside4'].sum()

8642.849999999986

In [119]:
df_ = final_df
spread = 2
for sl in [5,7,9,12,15,18,20,25]:
    print('SL',sl)
    loss = df_[df_['max_dd8'] >= sl - spread]
    win = df_[df_['max_dd8'] < sl - spread]
    print(loss.shape,win.shape)
    win_pips = round(win['max_upside8'].sum())
    loss_pips = round(loss.shape[0] * sl) + round(loss.shape[0] * spread)
    print('NET PIPS:',round(win_pips - loss_pips ),win_pips,loss_pips)

SL 5
(1259, 25) (151, 25)
NET PIPS: -316 8497 8813
SL 7
(1188, 25) (222, 25)
NET PIPS: 1336 12028 10692
SL 9
(1123, 25) (287, 25)
NET PIPS: 3063 15416 12353
SL 12
(1034, 25) (376, 25)
NET PIPS: 5230 19706 14476
SL 15
(942, 25) (468, 25)
NET PIPS: 7743 23757 16014
SL 18
(866, 25) (544, 25)
NET PIPS: 9886 27206 17320
SL 20
(815, 25) (595, 25)
NET PIPS: 11697 29627 17930
SL 25
(689, 25) (721, 25)
NET PIPS: 16056 34659 18603


In [104]:
df_ = final_df
spread = 2
for sl in [5,7,9,12,15,18,20,25]:
    print('SL',sl)
    loss = df_[df_['max_dd4'] >= sl - spread]
    win = df_[df_['max_dd4'] < sl - spread]
    print(loss.shape,win.shape)
    win_pips = round(win['max_upside4'].sum())
    loss_pips = round(loss.shape[0] * sl) + round(loss.shape[0] * spread)
    print('NET PIPS:',round(win_pips - loss_pips ),win_pips,loss_pips)

SL 5
(1207, 25) (203, 25)
NET PIPS: -163 8286 8449
SL 7
(1118, 25) (292, 25)
NET PIPS: 1468 11530 10062
SL 9
(1036, 25) (374, 25)
NET PIPS: 3339 14735 11396
SL 12
(928, 25) (482, 25)
NET PIPS: 5177 18169 12992
SL 15
(811, 25) (599, 25)
NET PIPS: 7962 21749 13787
SL 18
(720, 25) (690, 25)
NET PIPS: 10078 24478 14400
SL 20
(652, 25) (758, 25)
NET PIPS: 11884 26228 14344
SL 25
(519, 25) (891, 25)
NET PIPS: 15828 29841 14013


In [100]:
df_ = final_df
for sl in [5,7,9,12,15,18,20,25]:
    print('SL',sl)
    loss = df_[df_['max_dd4'] >= sl ]
    win = df_[df_['max_dd4'] < sl ]
    print(loss.shape,win.shape)
    win_pips = round(win['max_upside4'].sum())
    loss_pips = round(loss.shape[0] * sl) 
    print('NET PIPS:',round(win_pips - loss_pips ),win_pips,loss_pips)

SL 5
(511, 25) (153, 25)
NET PIPS: 2975 5530 2555
SL 7
(458, 25) (206, 25)
NET PIPS: 3709 6915 3206
SL 9
(409, 25) (255, 25)
NET PIPS: 4588 8269 3681
SL 12
(358, 25) (306, 25)
NET PIPS: 5193 9489 4296
SL 15
(310, 25) (354, 25)
NET PIPS: 6123 10773 4650
SL 18
(266, 25) (398, 25)
NET PIPS: 7087 11875 4788
SL 20
(236, 25) (428, 25)
NET PIPS: 7757 12477 4720
SL 25
(164, 25) (500, 25)
NET PIPS: 9731 13831 4100


In [ ]:
9731

In [93]:
df_.shape

(664, 25)

In [97]:
(((9731 * 2) / 10) / 12) / 4

40.545833333333334

In [90]:
for sl in [5,7,9,12,15,18,20,25]:
    print('SL',sl)
    loss = df_[df_['max_dd4'] >= sl]
    win = df_[df_['max_dd4'] < sl]
    print(loss.shape,win.shape)
    win_pips = round(win['max_upside4'].sum())
    loss_pips = round(loss.shape[0] * sl)
    print('NET PIPS:',round(win_pips - loss_pips ),win_pips,loss_pips)


SL 5
(52, 25) (25, 25)
NET PIPS: 678 938 260
SL 7
(50, 25) (27, 25)
NET PIPS: 656 1006 350
SL 9
(48, 25) (29, 25)
NET PIPS: 667 1099 432
SL 12
(37, 25) (40, 25)
NET PIPS: 966 1410 444
SL 15
(34, 25) (43, 25)
NET PIPS: 952 1462 510
SL 18
(27, 25) (50, 25)
NET PIPS: 1136 1622 486
SL 20
(25, 25) (52, 25)
NET PIPS: 1166 1666 500
SL 25
(19, 25) (58, 25)
NET PIPS: 1292 1767 475


In [82]:
new_df[['max_upside4','max_upside8']].mean()

max_upside4    14.035294
max_upside8    12.172549
dtype: float64

In [83]:
new_df[['max_upside4','max_upside8']].sum()

max_upside4    715.8
max_upside8    620.8
dtype: float64

In [72]:
df_[df_['next_eight_hours'] > 0][['same_hour_dd','next_hour_dd','next_two_hours_dd','next_three_hours_dd',
                                      'next_four_hours_dd','next_five_hours_dd','next_six_hours_dd','next_eight_hours_dd']].mean()

same_hour_dd            6.337255
next_hour_dd           -1.725490
next_two_hours_dd      -4.837255
next_three_hours_dd   -10.282353
next_four_hours_dd    -12.350980
next_five_hours_dd    -13.294118
next_six_hours_dd     -17.570588
next_eight_hours_dd   -22.533333
dtype: float64

In [73]:
new_df = df_[df_['next_eight_hours'] > 0][['same_hour_dd','next_hour_dd','next_two_hours_dd','next_three_hours_dd',
                                      'next_four_hours_dd','next_five_hours_dd','next_six_hours_dd','next_eight_hours_dd']]
new_df = new_df.sort_values(by = 'same_hour_dd',ascending = False)
new_df.head(20)

,same_hour_dd,next_hour_dd,next_two_hours_dd,next_three_hours_dd,next_four_hours_dd,next_five_hours_dd,next_six_hours_dd,next_eight_hours_dd
30,44.2,2.4,-112.8,-118.6,-118.4,-64.2,-69.8,10.0
12,33.5,26.6,55.7,34.6,24.7,28.1,8.8,-25.4
72,27.5,30.1,26.8,5.6,-14.0,-58.0,-60.4,-17.0
44,23.7,-2.3,-1.7,-6.7,-0.6,-4.8,1.6,-7.4
6,21.4,29.2,-2.5,9.0,12.5,49.7,44.8,-1.8
8,16.2,11.0,11.6,-5.4,-7.4,-16.4,24.2,-3.2
64,15.0,8.3,-9.2,-20.9,-7.1,-0.3,1.4,-9.3
1,13.1,9.0,13.8,9.9,8.6,14.0,19.0,-4.0
2,12.4,20.8,16.4,-0.6,-11.6,1.7,0.1,-20.6
51,12.0,1.2,-1.7,3.9,16.6,19.2,2.0,-3.7


In [74]:
new_df = df_[df_['next_eight_hours'] < 0][['same_hour_dd','next_hour_dd','next_two_hours_dd','next_three_hours_dd',
                                      'next_four_hours_dd','next_five_hours_dd','next_six_hours_dd','next_eight_hours_dd']]
new_df = new_df.sort_values(by = 'same_hour_dd',ascending = False)
new_df.head(20)

,same_hour_dd,next_hour_dd,next_two_hours_dd,next_three_hours_dd,next_four_hours_dd,next_five_hours_dd,next_six_hours_dd,next_eight_hours_dd
32,111.2,120.4,65.2,43.4,59.7,52.8,54.4,57.6
68,81.5,80.7,40.1,76.3,-18.4,-23.6,-67.5,73.0
31,38.0,149.2,158.4,103.2,81.4,97.7,90.8,80.0
11,37.6,71.1,64.2,93.3,72.2,62.3,65.7,23.9
25,26.7,20.8,13.1,14.2,28.6,50.3,41.6,48.8
33,25.3,16.1,-3.4,-9.2,54.8,64.6,41.8,62.0
23,20.7,12.9,8.5,0.1,-10.7,11.9,12.7,28.6
42,15.4,-1.0,-2.8,-2.0,-8.4,-4.1,1.8,10.6
71,14.4,22.8,25.1,23.4,20.5,40.2,48.2,21.6
26,14.4,36.1,27.4,40.8,34.6,27.7,43.2,37.9


In [59]:
def test_future(df,pair,column = 'support_indicator'):
    """
    THIS FUNCTION CHECKS ALL TRADES
    
    """
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i - 1,c_col]

            lst.append([mult * (arr[i ,c_col] - c),
                    mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 8,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['same_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours','next_eight_hours'])  

    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future2(df,pair,column = 'support_indicator'):
    """
    THIS FUNCTION CHECKS THE TRADES THAT START OFF NEGATIVE
    
    """
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] < 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  

    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def test_future3(df,pair,column = 'support_indicator'):
    """
    THIS FUNCTION CHECKS THE TRADES THAT START OFF POSITIVE
    
    """
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] > 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  

    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future4(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
        
    print('TESTING DRAWDOWN')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                        mult * (c - arr[i + 1,l_col]),
                        mult * (c - arr[i + 2,l_col] ),
                        mult * (c - arr[i + 3,l_col] ),
                        mult * (c - arr[i + 4,l_col] ), 
                        mult * (c - arr[i + 5,l_col] ), 
                        mult * (c - arr[i + 6,l_col] )
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  

    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def get_support(df,pair):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    delta = .0013
    delta2 = .008
    if 'JPY' in pair:
        delta = delta * 100
        delta2 = delta2 * 100

    
    arr = df.values
    for i in range(70,arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,o] - arr[i,c] >= delta and arr[i,d] > 0 and arr[i,c] <= arr[i - 75,c] - delta2:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def get_results(result_type = 1):
    all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
               'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
               'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
               'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
    medians = pd.DataFrame([-100])
    means = pd.DataFrame([-100])
    sums = pd.DataFrame([-100])
    for pair in all_pairs:


        print(pair,'LATER DATES')
        df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
        df = get_support(df,pair)

        df2 = load_df(pair = pair,granularity ='H1',start = datetime(2010,1,1,0,0,0),end = datetime(2016,1,1,0,0,0))
        df2 = get_support(df2,pair)
        if result_type == 1:
            print('RESULT TYPE 1 CHECKS THE FUTURE RESULTS OF ALL TRADES')
            test = test_future(df,pair,column = 'support_indicator')
            test2 = test_future(df2,pair,column = 'support_indicator')
        if result_type == 2:
            print('RESULT TYPE 2 CHECKS THE FUTURE RESULTS OF TRADES THAT START OFF NEGATIVE')
            test = test_future2(df,pair,column = 'support_indicator')
            test2 = test_future2(df2,pair,column = 'support_indicator') 
        if result_type == 3:
            print('RESULT TYPE 3 CHECKS THE FUTURE RESULTS OF TRADES THAT START OFF POSITIVE')
            test = test_future3(df,pair,column = 'support_indicator')
            test2 = test_future3(df2,pair,column = 'support_indicator')         
        if result_type == 4:
            print('RESULT TYPE 4 CHECKS THE DRAWDOWN OF TRADES')
            test = test_future4(df,pair,column = 'support_indicator')
            test2 = test_future4(df2,pair,column = 'support_indicator')    
        med1 = test[0]
        med2 = test2[0]
        med1 = med1.reset_index()
        med2 = med2.reset_index()
        med2.iloc[0,0] = med1.iloc[0,0] + '_early'
        med1.iloc[0,0] = med1.iloc[0,0] + '_late'
        temp_med = pd.concat([med1,med2])

        mean1 = test[1]
        mean2 = test2[1]
        mean1 = mean1.reset_index()
        mean2 = mean2.reset_index()
        mean2.iloc[0,0] = mean1.iloc[0,0] + '_early'
        mean1.iloc[0,0] = mean1.iloc[0,0] + '_late'
        temp_mean = pd.concat([mean1,mean2])

        sum1 = test[2]
        sum2 = test2[2]
        sum1 = sum1.reset_index()
        sum2 = sum2.reset_index()
        sum2.iloc[0,0] = sum1.iloc[0,0] + '_early'
        sum1.iloc[0,0] = sum1.iloc[0,0] + '_late'
        temp_sum = pd.concat([sum1,sum2])

        temp_med['shape'] = 0
        temp_mean['shape'] = 0
        temp_sum['shape'] = 0
        temp_med.iloc[0,-1] = shape1
        temp_med.iloc[1,-1] = shape2
        temp_mean.iloc[0,-1] = shape1
        temp_mean.iloc[1,-1] = shape2
        temp_sum.iloc[0,-1] = shape1
        temp_sum.iloc[1,-1] = shape2
        if medians.iloc[0,0] == -100:
            medians = temp_med
            means = temp_mean
            sums = temp_sum
        else:
            medians = pd.concat([medians,temp_med])
            means = pd.concat([means,temp_mean])
            sums = pd.concat([sums,temp_sum])
    print("MEDIANS")
    print(medians.mean())
    print()
    print("MEANS")
    print(means.mean())
    print()
    print("SUMS")
    print(sums.sum())
    print()
    print()
    print()
    print()
        
    return medians,means,sums
        
        
        
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
medians1,means1,sums1 = get_results(result_type = 1)
medians2,means2,sums2 = get_results(result_type = 2)
medians3,means3,sums3 = get_results(result_type = 3)
medians4,means4,sums4 = get_results(result_type = 4)

medians1


USD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_H1/2016-01-01_2022-07-31.csv
(41169, 15)
SHAPE (34, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_H1/2010-01-01_2016-01-01.csv
(39068, 15)
SHAPE (24, 16)
EUR_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2016-01-01_2022-07-31.csv
(41144, 15)
SHAPE (48, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2010-01-01_2016-01-01.csv
(39094, 15)
SHAPE (89, 16)
AUD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2016-01-01_2022-07-31.csv
(41163, 15)
SHAPE (29, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2010-01-01_2016-01-01.csv
(39082, 15)
SHAPE (61, 16)
CAD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2016-01-01_2022-07-31.csv
(41154, 15)
SHAPE (36, 16)
PAT

<ipython-input-59-9489ac7aea33>:221: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-59-9489ac7aea33>:224: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


(41169, 15)
SHAPE (34, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_H1/2010-01-01_2016-01-01.csv
(39068, 15)
SHAPE (24, 16)
EUR_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2016-01-01_2022-07-31.csv
(41144, 15)
SHAPE (48, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2010-01-01_2016-01-01.csv
(39094, 15)
SHAPE (89, 16)
AUD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2016-01-01_2022-07-31.csv
(41163, 15)
SHAPE (29, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2010-01-01_2016-01-01.csv
(39082, 15)
SHAPE (61, 16)
CAD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2016-01-01_2022-07-31.csv
(41154, 15)
SHAPE (36, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2010-01-01_2016-01-01.csv
(39049, 15)
SHAPE (64, 

<ipython-input-59-9489ac7aea33>:221: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-59-9489ac7aea33>:224: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


(41169, 15)
SHAPE (34, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_H1/2010-01-01_2016-01-01.csv
(39068, 15)
SHAPE (24, 16)
EUR_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2016-01-01_2022-07-31.csv
(41144, 15)
SHAPE (48, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2010-01-01_2016-01-01.csv
(39094, 15)
SHAPE (89, 16)
AUD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2016-01-01_2022-07-31.csv
(41163, 15)
SHAPE (29, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2010-01-01_2016-01-01.csv
(39082, 15)
SHAPE (61, 16)
CAD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2016-01-01_2022-07-31.csv
(41154, 15)
SHAPE (36, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2010-01-01_2016-01-01.csv
(39049, 15)
SHAPE (64, 

<ipython-input-59-9489ac7aea33>:221: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-59-9489ac7aea33>:224: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


(41169, 15)
SHAPE (34, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_H1/2010-01-01_2016-01-01.csv
(39068, 15)
SHAPE (24, 16)
TESTING DRAWDOWN
TESTING DRAWDOWN
EUR_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2016-01-01_2022-07-31.csv
(41144, 15)
SHAPE (48, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_H1/2010-01-01_2016-01-01.csv
(39094, 15)
SHAPE (89, 16)
TESTING DRAWDOWN
TESTING DRAWDOWN
AUD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2016-01-01_2022-07-31.csv
(41163, 15)
SHAPE (29, 16)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_JPY_H1/2010-01-01_2016-01-01.csv
(39082, 15)
SHAPE (61, 16)
TESTING DRAWDOWN
TESTING DRAWDOWN
CAD_JPY LATER DATES
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_H1/2016-01-01_2022-07-31.csv
(41154, 15)
SHAPE (36, 16)
PATH: /Users/aidanmcconne

<ipython-input-59-9489ac7aea33>:221: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-59-9489ac7aea33>:224: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


,index,same_hour,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours,next_eight_hours,shape
0,USD_JPY_late,1.70,8.15,5.45,5.50,4.40,6.00,0.55,10.70,12
0,USD_JPY_early,4.80,3.35,6.70,7.30,8.10,4.20,8.00,8.15,77
0,EUR_JPY_late,0.50,1.60,2.80,4.85,7.80,7.90,7.55,15.70,12
0,EUR_JPY_early,-0.80,1.50,1.30,8.00,5.80,5.80,8.30,9.10,77
0,AUD_JPY_late,-1.70,4.80,-0.40,6.70,4.80,2.90,-4.70,-2.60,12
0,AUD_JPY_early,-1.00,-0.30,-4.00,-4.80,-5.00,-4.70,-1.40,-2.40,77
0,CAD_JPY_late,-0.50,3.65,3.55,10.05,10.10,5.00,-0.90,2.25,12
0,CAD_JPY_early,2.45,4.55,4.20,6.65,4.90,6.05,4.80,10.30,77
0,CHF_JPY_late,-0.70,1.30,2.25,0.80,5.35,3.65,6.55,8.40,12
0,CHF_JPY_early,1.10,7.45,9.00,10.40,11.05,10.45,12.45,11.45,77


In [56]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            1.67625
next_two_hours       2.24625
next_three_hours     2.50875
next_four_hours      2.34250
next_five_hours      2.82375
next_six_hours       2.90000
next_eight_hours     4.36000
shape               51.90000
dtype: float64

MEANS
next_hour            1.536683
next_two_hours       2.348554
next_three_hours     2.544410
next_four_hours      2.630305
next_five_hours      2.380456
next_six_hours       2.216045
next_eight_hours     3.618501
shape               51.900000
dtype: float64

SUMS
index               USD_JPY_lateUSD_JPY_earlyEUR_JPY_lateEUR_JPY_e...
next_hour                                                      1866.0
next_two_hours                                                 3694.4
next_three_hours                                               3845.7
next_four_hours                                                3857.7
next_five_hours                                                3723.3
next_six_hours                                           

<ipython-input-56-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-56-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [54]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            1.81625
next_two_hours       2.12375
next_three_hours     2.12375
next_four_hours      1.22125
next_five_hours      2.73625
next_six_hours       2.39875
next_eight_hours     3.54625
shape               33.80000
dtype: float64

MEANS
next_hour            1.845411
next_two_hours       2.282673
next_three_hours     2.086070
next_four_hours      2.361634
next_five_hours      2.182387
next_six_hours       1.721435
next_eight_hours     2.814809
shape               33.800000
dtype: float64

SUMS
index               USD_JPY_lateUSD_JPY_earlyEUR_JPY_lateEUR_JPY_e...
next_hour                                                      1088.3
next_two_hours                                                 2185.2
next_three_hours                                               1793.7
next_four_hours                                                2234.6
next_five_hours                                                2191.5
next_six_hours                                           

<ipython-input-54-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-54-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [52]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            1.86500
next_two_hours       2.35375
next_three_hours     2.38500
next_four_hours      1.97875
next_five_hours      2.96250
next_six_hours       2.42750
next_eight_hours     3.66625
shape               43.27500
dtype: float64

MEANS
next_hour            1.794629
next_two_hours       2.633860
next_three_hours     2.522136
next_four_hours      2.649872
next_five_hours      2.348115
next_six_hours       1.746037
next_eight_hours     2.835029
shape               43.275000
dtype: float64

SUMS
index               USD_JPY_lateUSD_JPY_earlyEUR_JPY_lateEUR_JPY_e...
next_hour                                                      1743.6
next_two_hours                                                 3349.6
next_three_hours                                               2888.0
next_four_hours                                                2997.6
next_five_hours                                                2649.1
next_six_hours                                           

<ipython-input-52-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-52-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [50]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            2.183333
next_two_hours       2.191667
next_three_hours     2.013333
next_four_hours      1.738333
next_five_hours      2.596667
next_six_hours       3.143333
next_eight_hours     3.945000
shape               44.433333
dtype: float64

MEANS
next_hour            2.201794
next_two_hours       2.861436
next_three_hours     2.694921
next_four_hours      2.903933
next_five_hours      3.353449
next_six_hours       3.331123
next_eight_hours     3.832830
shape               44.433333
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                      1806.2
next_two_hours                                                 2835.9
next_three_hours                                               2291.9
next_four_hours                                                2415.5
next_five_hours                                                3022.2
next_six_hours                                   

<ipython-input-50-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-50-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [44]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            0.138333
next_two_hours       0.326667
next_three_hours    -0.368333
next_four_hours     -0.185000
next_five_hours     -0.341667
next_six_hours       0.396667
next_eight_hours    -1.710000
shape               28.066667
dtype: float64

MEANS
next_hour           -0.030668
next_two_hours      -0.400605
next_three_hours    -1.362240
next_four_hours     -0.568689
next_five_hours     -0.401767
next_six_hours      -0.175163
next_eight_hours    -1.511287
shape               28.066667
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                        41.8
next_two_hours                                                 -336.9
next_three_hours                                              -1580.8
next_four_hours                                                -949.3
next_five_hours                                               -1035.5
next_six_hours                                   

<ipython-input-44-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-44-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [42]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            0.223333
next_two_hours       0.075000
next_three_hours    -0.366667
next_four_hours     -0.518333
next_five_hours      0.418333
next_six_hours      -0.065000
next_eight_hours    -0.990000
shape               20.466667
dtype: float64

MEANS
next_hour           -0.288615
next_two_hours       0.168623
next_three_hours    -0.012637
next_four_hours      0.152158
next_five_hours      0.191139
next_six_hours       0.129313
next_eight_hours    -1.545948
shape               20.466667
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                      -235.2
next_two_hours                                                  -84.7
next_three_hours                                                 73.6
next_four_hours                                                 -58.9
next_five_hours                                                 107.5
next_six_hours                                   

<ipython-input-42-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-42-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [40]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            1.235000
next_two_hours       0.460000
next_three_hours     0.985000
next_four_hours      0.190000
next_five_hours      0.778333
next_six_hours       1.226667
next_eight_hours    -0.708333
shape               47.900000
dtype: float64

MEANS
next_hour            0.599796
next_two_hours      -0.283232
next_three_hours    -0.256054
next_four_hours     -1.193467
next_five_hours     -0.560878
next_six_hours      -0.362792
next_eight_hours    -0.983025
shape               47.900000
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                      1067.3
next_two_hours                                                 -238.5
next_three_hours                                                -92.3
next_four_hours                                               -1649.1
next_five_hours                                               -1209.9
next_six_hours                                   

<ipython-input-40-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-40-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [35]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            2.151667
next_two_hours       2.516667
next_three_hours     2.566667
next_four_hours      1.516667
next_five_hours      2.313333
next_six_hours       2.445000
next_eight_hours     3.138333
shape               51.633333
dtype: float64

MEANS
next_hour            2.465162
next_two_hours       3.128161
next_three_hours     2.787135
next_four_hours      2.897023
next_five_hours      3.134282
next_six_hours       3.180199
next_eight_hours     3.348491
shape               51.633333
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                      2531.6
next_two_hours                                                 3950.9
next_three_hours                                               3350.7
next_four_hours                                                3528.9
next_five_hours                                                3511.1
next_six_hours                                   

<ipython-input-35-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-35-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [33]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            1.300000
next_two_hours       0.513333
next_three_hours     0.911667
next_four_hours      0.758333
next_five_hours      1.165000
next_six_hours       1.071667
next_eight_hours     2.641667
shape               37.633333
dtype: float64

MEANS
next_hour            0.945360
next_two_hours       0.883084
next_three_hours     1.217327
next_four_hours      1.797379
next_five_hours      2.350447
next_six_hours       2.518419
next_eight_hours     3.316851
shape               37.633333
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                      1349.1
next_two_hours                                                 1473.2
next_three_hours                                               1632.0
next_four_hours                                                2113.7
next_five_hours                                                2515.4
next_six_hours                                   

<ipython-input-33-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-33-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [31]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            1.220000
next_two_hours       1.038333
next_three_hours     1.250000
next_four_hours      0.533333
next_five_hours      2.241667
next_six_hours       1.535000
next_eight_hours     3.033333
shape               22.633333
dtype: float64

MEANS
next_hour            1.138551
next_two_hours       1.244711
next_three_hours     1.438003
next_four_hours      2.452745
next_five_hours      3.072248
next_six_hours       3.451892
next_eight_hours     3.957850
shape               22.633333
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                       916.5
next_two_hours                                                 1078.7
next_three_hours                                                992.1
next_four_hours                                                1490.1
next_five_hours                                                1812.5
next_six_hours                                   

<ipython-input-31-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-31-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [27]:
print("MEDIANS")
print(medians.mean())
print()
print("MEANS")
print(means.mean())
print()
print("SUMS")
print(sums.sum())
print()

MEDIANS
next_hour            1.276667
next_two_hours       0.990000
next_three_hours     1.671667
next_four_hours      1.706667
next_five_hours      1.391667
next_six_hours       1.453333
next_eight_hours     2.405000
shape               35.566667
dtype: float64

MEANS
next_hour            1.173552
next_two_hours       1.629998
next_three_hours     1.526164
next_four_hours      1.794275
next_five_hours      2.208774
next_six_hours       2.282987
next_eight_hours     2.197237
shape               35.566667
dtype: float64

SUMS
index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                      1236.3
next_two_hours                                                 1692.3
next_three_hours                                               1321.6
next_four_hours                                                1526.0
next_five_hours                                                1966.4
next_six_hours                                   

<ipython-input-27-f8c78cd7f8fa>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(medians.mean())
<ipython-input-27-f8c78cd7f8fa>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(means.mean())


In [24]:
medians.mean()

<ipython-input-24-edf68f150aaa>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  medians.mean()


next_hour            0.931667
next_two_hours       1.221667
next_three_hours     1.523333
next_four_hours      1.480000
next_five_hours      1.683333
next_six_hours       2.281667
next_eight_hours     2.433333
shape               88.533333
dtype: float64

In [22]:
means.mean()

<ipython-input-22-bef750cc6f94>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  means.mean()


next_hour            1.230549
next_two_hours       2.031344
next_three_hours     1.864802
next_four_hours      2.006982
next_five_hours      2.379906
next_six_hours       2.825985
next_eight_hours     2.559038
shape               88.533333
dtype: float64

In [21]:
sums.sum()

index               GBP_USD_lateGBP_USD_earlyUSD_CAD_lateUSD_CAD_e...
next_hour                                                      2537.6
next_two_hours                                                 5010.0
next_three_hours                                               4184.2
next_four_hours                                                4312.1
next_five_hours                                                5329.4
next_six_hours                                                 6238.7
next_eight_hours                                               5750.7
shape                                                            2656
dtype: object

In [17]:
sums

,index,next_hour,next_two_hours,next_three_hours,next_four_hours,next_five_hours,next_six_hours,shape
0,GBP_USD_late,18.3,75.6,-52.1,-98.8,10.6,71.7,88
0,GBP_USD_early,248.2,424.5,430.7,95.8,357.6,677.8,184
0,USD_CAD_late,262.8,243.3,356.1,375.4,420.6,421.2,86
0,USD_CAD_early,75.4,135.9,147.4,143.5,124.8,202.7,84
0,USD_CHF_late,127.6,112.4,131.5,163.4,165.3,195.4,49
0,USD_CHF_early,177.1,340.9,202.7,112.6,257.3,60.8,114
0,EUR_USD_late,85.2,292.4,301.4,394.1,305.6,475.1,47
0,EUR_USD_early,-50.6,16.0,280.1,409.9,449.9,486.7,96
0,AUD_USD_late,93.9,16.4,-10.1,-91.2,-65.9,-5.2,64
0,AUD_USD_early,134.5,857.1,306.1,538.5,710.0,546.5,130


In [38]:
100 / 24

4.166666666666667

In [39]:
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future2(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] < 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def test_future3(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] > 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future4(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
        
    print('TESTING DRAWDOWN')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] > 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                        mult * (c - arr[i + 1,l_col]),
                        mult * (c - arr[i + 2,l_col] ),
                        mult * (c - arr[i + 3,l_col] ),
                        mult * (c - arr[i + 4,l_col] ), 
                        mult * (c - arr[i + 5,l_col] ), 
                        mult * (c - arr[i + 6,l_col] )
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def get_support(df):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    
    arr = df.values
    for i in range(100,arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,o] - arr[i,c] >= .001 and arr[i,d] > 0 and arr[i,c] >= arr[i - 80 ,c] + .004:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
for pair in all_pairs:
    if 'JPY' in pair:
        pass
    else:
        print(pair)
    #pair = 'EUR_USD'
        df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
        df = get_support(df)
        test = test_future(df,pair,column = 'support_indicator')
        test = test_future2(df,pair,column = 'support_indicator')
        test = test_future3(df,pair,column = 'support_indicator')
        test = test_future4(df,pair,column = 'support_indicator')


GBP_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_USD_H1/2016-01-01_2022-07-31.csv
(41155, 15)
SHAPE (68, 16)
MEDIAN:
next_hour          -0.25
next_two_hours     -2.45
next_three_hours   -3.20
next_four_hours     0.10
next_five_hours     0.20
next_six_hours     -0.60
dtype: float64

MEAN:
next_hour           0.497059
next_two_hours     -0.280882
next_three_hours   -0.882353
next_four_hours     0.198529
next_five_hours     1.060294
next_six_hours     -1.241176
dtype: float64

SUM:
   next_hour  next_two_hours  next_three_hours  next_four_hours  \
0       33.8           -19.1             -60.0             13.5   

   next_five_hours  next_six_hours  
0             72.1           -84.4  
None
MEDIAN:
current_hour       -9.2
next_hour          -5.7
next_two_hours     -8.3
next_three_hours   -6.2
next_four_hours    -4.2
next_five_hours     1.6
next_six_hours     -7.6
dtype: float64

MEAN:
current_hour       -10.152
next_hour          -10.068
next_two_hours     -

In [31]:
150 / 24

6.25

In [27]:
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future2(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] < 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def test_future3(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    o_col = df.columns.get_loc('o')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] > 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                mult * (arr[i + 1,c_col] - c),
                        mult * (arr[i + 2,c_col] - c),
                        mult * (arr[i + 3,c_col] - c),
                        mult * (arr[i + 4,c_col] - c), 
                        mult * (arr[i + 5,c_col] - c), 
                        mult * (arr[i + 6,c_col] - c)
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
def test_future4(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
        
    print('TESTING DRAWDOWN')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    col = df.columns.get_loc(column)
    dv_col = df.columns.get_loc('delta_vector')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,dv_col] > 0:
            c = arr[i,o_col]

            lst.append([mult * (arr[i ,c_col] - c),
                        mult * (c - arr[i + 1,l_col]),
                        mult * (c - arr[i + 2,l_col] ),
                        mult * (c - arr[i + 3,l_col] ),
                        mult * (c - arr[i + 4,l_col] ), 
                        mult * (c - arr[i + 5,l_col] ), 
                        mult * (c - arr[i + 6,l_col] )
                       ])
    df2 = pd.DataFrame(lst,columns = ['current_hour','next_hour','next_two_hours','next_three_hours','next_four_hours','next_five_hours','next_six_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def get_support(df):
    c = df.columns.get_loc('c')
    o = df.columns.get_loc('o')
    l = df.columns.get_loc('l')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d = df.columns.get_loc('delta')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,c] == arr[i,l] and arr[i,o] - arr[i,c] >= 0 and arr[i,c] <= arr[i - 40,c] - .004:
            arr[i + 1,new_col] = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df



pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))

print('SHAPE',df.shape)
df = get_support(df)
test = test_future(df,pair,column = 'support_indicator')
test = test_future2(df,pair,column = 'support_indicator')
test = test_future3(df,pair,column = 'support_indicator')
test = test_future4(df,pair,column = 'support_indicator')



PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_H4/2016-01-01_2022-07-31.csv
(10661, 15)
SHAPE (10661, 15)
SHAPE (25, 16)
MEDIAN:
next_hour           -0.1
next_two_hours       2.7
next_three_hours    14.6
next_four_hours     12.4
next_five_hours     13.5
next_six_hours      15.3
dtype: float64

MEAN:
next_hour            1.356
next_two_hours       7.948
next_three_hours    14.352
next_four_hours     24.304
next_five_hours     23.416
next_six_hours      19.600
dtype: float64

SUM:
   next_hour  next_two_hours  next_three_hours  next_four_hours  \
0       33.9           198.7             358.8            607.6   

   next_five_hours  next_six_hours  
0            585.4           490.0  
None
MEDIAN:
current_hour        -5.3
next_hour          -10.0
next_two_hours      -2.9
next_three_hours    11.5
next_four_hours      4.3
next_five_hours     15.0
next_six_hours      16.8
dtype: float64

MEAN:
current_hour       -10.700000
next_hour           -3.461538
next_two_

In [22]:
pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='H1',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df2 = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df2 = df2[['time','o','h','l','c']]
df2.columns = ['time','o4','h4','l4','c4']
df = df.merge(df2,how = 'left',on = 'time')
df.head(30)

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_H1/2016-01-01_2022-07-31.csv
(41152, 15)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_H4/2016-01-01_2022-07-31.csv
(10661, 15)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,hour_col,xau_morning,o4,h4,l4,c4
0,2016-01-03T22:00:00.000000000Z,289,1.08743,1.08746,1.08667,1.0867,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.00073,0.00079,0.00003,0.00003,0.00073,22,0,1.08743,1.08746,1.08271,1.08597
1,2016-01-03T23:00:00.000000000Z,911,1.08665,1.0867,1.08454,1.08519,2016-01-03 23:00:00+00:00,2016-01-03 18:00:00-05:00,-0.00146,0.00216,0.00005,0.00065,0.00146,23,0,NaN,NaN,NaN,NaN
2,2016-01-04T00:00:00.000000000Z,794,1.08524,1.08578,1.08432,1.08454,2016-01-04 00:00:00+00:00,2016-01-03 19:00:00-05:00,-0.0007,0.00146,0.00054,0.00022,0.0007,0,0,NaN,NaN,NaN,NaN
3,2016-01-04T01:00:00.000000000Z,1565,1.08456,1.08617,1.08271,1.08597,2016-01-04 01:00:00+00:00,2016-01-03 20:00:00-05:00,0.00141,0.00346,0.0002,0.00185,0.00141,1,0,NaN,NaN,NaN,NaN
4,2016-01-04T02:00:00.000000000Z,1728,1.086,1.08757,1.08586,1.08716,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,0.00116,0.00171,0.00041,0.00014,0.00116,2,0,1.086,1.09048,1.08586,1.08972
5,2016-01-04T03:00:00.000000000Z,610,1.08718,1.0873,1.08671,1.08713,2016-01-04 03:00:00+00:00,2016-01-03 22:00:00-05:00,-0.00005,0.00059,0.00012,0.00042,0.00005,3,0,NaN,NaN,NaN,NaN
6,2016-01-04T04:00:00.000000000Z,984,1.08717,1.08836,1.08697,1.08793,2016-01-04 04:00:00+00:00,2016-01-03 23:00:00-05:00,0.00076,0.00139,0.00043,0.0002,0.00076,4,0,NaN,NaN,NaN,NaN
7,2016-01-04T05:00:00.000000000Z,1501,1.0879,1.09048,1.08739,1.08972,2016-01-04 05:00:00+00:00,2016-01-04 00:00:00-05:00,0.00182,0.00309,0.00076,0.00051,0.00182,5,0,NaN,NaN,NaN,NaN
8,2016-01-04T06:00:00.000000000Z,1073,1.0897,1.09024,1.08842,1.08888,2016-01-04 06:00:00+00:00,2016-01-04 01:00:00-05:00,-0.00082,0.00182,0.00054,0.00046,0.00082,6,0,1.0897,1.09464,1.08818,1.09292
9,2016-01-04T07:00:00.000000000Z,3110,1.08891,1.09204,1.08818,1.09127,2016-01-04 07:00:00+00:00,2016-01-04 02:00:00-05:00,0.00236,0.00386,0.00077,0.00073,0.00236,7,0,NaN,NaN,NaN,NaN


In [13]:
df3 = df.merge(df2,how = 'left',on = 'time')
df3 = df3.fillna(0)
df3.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,hour_col,xau_morning,o4,h4,l4,c4
0,2016-01-03T22:00:00.000000000Z,289,1.08743,1.08746,1.08667,1.08670,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.00073,0.00079,0.00003,0.00003,0.00073,22,0,1.08743,1.08746,1.08271,1.08597
1,2016-01-03T23:00:00.000000000Z,911,1.08665,1.08670,1.08454,1.08519,2016-01-03 23:00:00+00:00,2016-01-03 18:00:00-05:00,-0.00146,0.00216,0.00005,0.00065,0.00146,23,0,0.00000,0.00000,0.00000,0.00000
2,2016-01-04T00:00:00.000000000Z,794,1.08524,1.08578,1.08432,1.08454,2016-01-04 00:00:00+00:00,2016-01-03 19:00:00-05:00,-0.00070,0.00146,0.00054,0.00022,0.00070,0,0,0.00000,0.00000,0.00000,0.00000
3,2016-01-04T01:00:00.000000000Z,1565,1.08456,1.08617,1.08271,1.08597,2016-01-04 01:00:00+00:00,2016-01-03 20:00:00-05:00,0.00141,0.00346,0.00020,0.00185,0.00141,1,0,0.00000,0.00000,0.00000,0.00000
4,2016-01-04T02:00:00.000000000Z,1728,1.08600,1.08757,1.08586,1.08716,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,0.00116,0.00171,0.00041,0.00014,0.00116,2,0,1.08600,1.09048,1.08586,1.08972


In [8]:
df3.shape

(41160, 19)

In [9]:
df.shape

(41152, 15)

In [10]:
df2.shape

(10661, 15)